In [1]:
!pip install torch torchvision torchaudio
!pip install ultralytics
!pip install albumentations

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.3 MB/s eta 0:00:00
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.6/865.6 kB 7.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 8.6 MB/s eta 0:00:0000:0100:01
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 7.2 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 6.6 MB/s eta 0:00:00:00:01


In [3]:
import os
import json
import cv2
import numpy as np
from ultralytics import YOLO
from sklearn.model_selection import train_test_split
from albumentations import (
    Compose, RandomBrightnessContrast, HueSaturationValue, GaussNoise,
    HorizontalFlip, RandomCrop, Rotate, ShiftScaleRotate
)

# 데이터 로딩 및 전처리 함수
def load_and_preprocess_data(json_path, image_dir):
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    image_path = os.path.join(image_dir, data['image_name'])
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    annotations = data['Annotation']
    boxes = []
    labels = []
    for ann in annotations:
        boxes.append(ann['data'])
        labels.append(ann['class_name'])
    
    return image, np.array(boxes), labels

# 박스 정규화 함수
def normalize_boxes(boxes, image_size):
    return boxes / np.array([image_size[1], image_size[0], image_size[1], image_size[0]])

# 데이터 증강 함수
def augment_data(image, boxes, labels):
    augmentations = Compose([
        RandomBrightnessContrast(p=0.5),
        HueSaturationValue(p=0.5),
        GaussNoise(p=0.3),
        HorizontalFlip(p=0.5),
        RandomCrop(height=image.shape[0], width=image.shape[1], p=0.3),
        Rotate(limit=10, p=0.3),
        ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=10, p=0.3)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})
    
    augmented = augmentations(image=image, bboxes=boxes, labels=labels)
    return augmented['image'], np.array(augmented['bboxes']), augmented['labels']

# YOLO 형식으로 레이블 변환
def convert_to_yolo_format(boxes, labels, image_size):
    yolo_labels = []
    for box, label in zip(boxes, labels):
        x_center = (box[0] + box[2]) / 2 / image_size[1]
        y_center = (box[1] + box[3]) / 2 / image_size[0]
        width = (box[2] - box[0]) / image_size[1]
        height = (box[3] - box[1]) / image_size[0]
        class_id = class_mapping[label]
        yolo_labels.append(f"{class_id} {x_center} {y_center} {width} {height}")
    return yolo_labels

# 데이터셋 준비 함수
def prepare_dataset(json_dir, image_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'images'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'labels'), exist_ok=True)

    data = []
    for json_file in os.listdir(json_dir):
        if json_file.endswith('.json'):
            json_path = os.path.join(json_dir, json_file)
            image, boxes, labels = load_and_preprocess_data(json_path, image_dir)
            
            # 데이터 증강
            aug_image, aug_boxes, aug_labels = augment_data(image, boxes, labels)
            
            # YOLO 형식으로 변환
            yolo_labels = convert_to_yolo_format(aug_boxes, aug_labels, aug_image.shape[:2])
            
            # 이미지 저장
            image_filename = f"aug_{json_file[:-5]}.jpg"
            cv2.imwrite(os.path.join(output_dir, 'images', image_filename), cv2.cvtColor(aug_image, cv2.COLOR_RGB2BGR))
            
            # 레이블 저장
            label_filename = f"aug_{json_file[:-5]}.txt"
            with open(os.path.join(output_dir, 'labels', label_filename), 'w') as f:
                f.write('\n'.join(yolo_labels))
            
            data.append(image_filename)
    
    return data

# 클래스 매핑 정의
class_mapping = {
    'car': 0, 'bus': 1, 'truck': 2, 'special vehicle': 3,
    'motorcycle': 4, 'bicycle': 5, 'personal mobility': 6,
    'person': 7, 'Traffic_light': 8, 'Traffic_sign': 9
}

# 메인 실행 코드
if __name__ == "__main__":
    # 데이터 준비
    train_data = prepare_dataset('./training/labels', './training/images', './training//output/train')
    val_data = prepare_dataset('./validation/labels', './validation/images', './validation/output/val')

    # 데이터 분할
    train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

    # YAML 설정 파일 생성
    yaml_content = f"""
    train: ./training/images
    val: ./validation/images

    nc: {len(class_mapping)}
    names: {list(class_mapping.keys())}
    """

    with open('dataset.yaml', 'w') as f:
        f.write(yaml_content)

    # 모델 로드 및 학습
    model = YOLO('yolov5s.pt')  # 사전 학습된 YOLOv5s 모델 로드

    # 모델 학습
    results = model.train(
        data='dataset.yaml',
        epochs=100,
        imgsz=640,
        batch=16,
        name='yolov5_autonomous_driving'
    )

    # 모델 평가
    results = model.val()

    # 테스트 데이터에 대한 예측
    test_results = model('./test/images')

    # 결과 저장
    test_results.save('./save/results')

[ WARN:0@749.230] global loadsave.cpp:241 findDecoder imread_('./training/images/S_DRG_230629_008_FC_049.jpg'): can't open/read file: check file path/integrity


error: OpenCV(4.10.0) /io/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [4]:
import os
import json
import cv2
import numpy as np
from ultralytics import YOLO
from sklearn.model_selection import train_test_split
from albumentations import (
    Compose, RandomBrightnessContrast, HueSaturationValue, GaussNoise,
    HorizontalFlip, RandomCrop, Rotate, ShiftScaleRotate
)

# 데이터 로딩 및 전처리 함수
def load_and_preprocess_data(json_path, image_dir):
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    image_path = os.path.join(image_dir, data['image_name'])
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    annotations = data['Annotation']
    boxes = []
    labels = []
    for ann in annotations:
        boxes.append(ann['data'])
        labels.append(ann['class_name'])
    
    return image, np.array(boxes), labels

# 박스 정규화 함수
def normalize_boxes(boxes, image_size):
    return boxes / np.array([image_size[1], image_size[0], image_size[1], image_size[0]])

# 데이터 증강 함수
def augment_data(image, boxes, labels):
    augmentations = Compose([
        RandomBrightnessContrast(p=0.5),
        HueSaturationValue(p=0.5),
        GaussNoise(p=0.3),
        HorizontalFlip(p=0.5),
        RandomCrop(height=image.shape[0], width=image.shape[1], p=0.3),
        Rotate(limit=10, p=0.3),
        ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=10, p=0.3)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})
    
    augmented = augmentations(image=image, bboxes=boxes, labels=labels)
    return augmented['image'], np.array(augmented['bboxes']), augmented['labels']

# YOLO 형식으로 레이블 변환
def convert_to_yolo_format(boxes, labels, image_size):
    yolo_labels = []
    for box, label in zip(boxes, labels):
        x_center = (box[0] + box[2]) / 2 / image_size[1]
        y_center = (box[1] + box[3]) / 2 / image_size[0]
        width = (box[2] - box[0]) / image_size[1]
        height = (box[3] - box[1]) / image_size[0]
        class_id = class_mapping[label]
        yolo_labels.append(f"{class_id} {x_center} {y_center} {width} {height}")
    return yolo_labels

# 데이터셋 준비 함수
def prepare_dataset(data_dir):
    images_dir = os.path.join(data_dir, 'images')
    labels_dir = os.path.join(data_dir, 'labels')
    
    data = []
    for json_file in os.listdir(labels_dir):
        if json_file.endswith('.json'):
            json_path = os.path.join(labels_dir, json_file)
            image, boxes, labels = load_and_preprocess_data(json_path, images_dir)
            
            # 데이터 증강
            aug_image, aug_boxes, aug_labels = augment_data(image, boxes, labels)
            
            # YOLO 형식으로 변환
            yolo_labels = convert_to_yolo_format(aug_boxes, aug_labels, aug_image.shape[:2])
            
            # 증강된 이미지 저장
            aug_image_filename = f"aug_{json_file[:-5]}.jpg"
            cv2.imwrite(os.path.join(images_dir, aug_image_filename), cv2.cvtColor(aug_image, cv2.COLOR_RGB2BGR))
            
            # 증강된 레이블 저장
            aug_label_filename = f"aug_{json_file[:-5]}.txt"
            with open(os.path.join(labels_dir, aug_label_filename), 'w') as f:
                f.write('\n'.join(yolo_labels))
            
            data.append(aug_image_filename)
    
    return data

# 클래스 매핑 정의
class_mapping = {
    'car': 0, 'bus': 1, 'truck': 2, 'special vehicle': 3,
    'motorcycle': 4, 'bicycle': 5, 'personal mobility': 6,
    'person': 7, 'Traffic_light': 8, 'Traffic_sign': 9
}

# 메인 실행 코드
if __name__ == "__main__":
    # 데이터 준비
    train_data = prepare_dataset('training')
    val_data = prepare_dataset('validation')

    # YAML 설정 파일 생성
    yaml_content = f"""
    train: training/images
    val: validation/images
    test: test/images

    nc: {len(class_mapping)}
    names: {list(class_mapping.keys())}
    """

    with open('dataset.yaml', 'w') as f:
        f.write(yaml_content)

    # 모델 로드 및 학습
    model = YOLO('yolov5s.pt')  # 사전 학습된 YOLOv5s 모델 로드

    # 모델 학습
    results = model.train(
        data='dataset.yaml',
        epochs=100,
        imgsz=640,
        batch=16,
        name='yolov5_autonomous_driving'
    )

    # 모델 평가
    results = model.val()

    # 테스트 데이터에 대한 예측
    test_results = model('test/images')

    # 결과 저장
    test_results.save('test_results')

[ WARN:0@6013.570] global loadsave.cpp:241 findDecoder imread_('training/images/S_DRG_230629_008_FC_049.jpg'): can't open/read file: check file path/integrity


error: OpenCV(4.10.0) /io/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [5]:
import os
import json
import cv2
import numpy as np
from ultralytics import YOLO
from sklearn.model_selection import train_test_split
from albumentations import (
    Compose, RandomBrightnessContrast, HueSaturationValue, GaussNoise,
    HorizontalFlip, RandomCrop, Rotate, ShiftScaleRotate
)

# 데이터 로딩 및 전처리 함수
def load_and_preprocess_data(json_path, image_dir):
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    image_path = os.path.join(image_dir, data['image_name'])
    
    # 파일 존재 여부 확인
    if not os.path.exists(image_path):
        print(f"Image file not found: {image_path}")
        return None, None, None
    
    image = cv2.imread(image_path)
    
    # 이미지 로드 실패 확인
    if image is None:
        print(f"Failed to load image: {image_path}")
        return None, None, None
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    annotations = data['Annotation']
    boxes = []
    labels = []
    for ann in annotations:
        boxes.append(ann['data'])
        labels.append(ann['class_name'])
    
    return image, np.array(boxes), labels

# 박스 정규화 함수
def normalize_boxes(boxes, image_size):
    return boxes / np.array([image_size[1], image_size[0], image_size[1], image_size[0]])

# 데이터 증강 함수
def augment_data(image, boxes, labels):
    augmentations = Compose([
        RandomBrightnessContrast(p=0.5),
        HueSaturationValue(p=0.5),
        GaussNoise(p=0.3),
        HorizontalFlip(p=0.5),
        RandomCrop(height=image.shape[0], width=image.shape[1], p=0.3),
        Rotate(limit=10, p=0.3),
        ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=10, p=0.3)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})
    
    augmented = augmentations(image=image, bboxes=boxes, labels=labels)
    return augmented['image'], np.array(augmented['bboxes']), augmented['labels']

# YOLO 형식으로 레이블 변환
def convert_to_yolo_format(boxes, labels, image_size):
    yolo_labels = []
    for box, label in zip(boxes, labels):
        x_center = (box[0] + box[2]) / 2 / image_size[1]
        y_center = (box[1] + box[3]) / 2 / image_size[0]
        width = (box[2] - box[0]) / image_size[1]
        height = (box[3] - box[1]) / image_size[0]
        class_id = class_mapping[label]
        yolo_labels.append(f"{class_id} {x_center} {y_center} {width} {height}")
    return yolo_labels

# 데이터셋 준비 함수
def prepare_dataset(data_dir):
    images_dir = os.path.join(data_dir, 'images')
    labels_dir = os.path.join(data_dir, 'labels')
    
    data = []
    for json_file in os.listdir(labels_dir):
        if json_file.endswith('.json'):
            json_path = os.path.join(labels_dir, json_file)
            image, boxes, labels = load_and_preprocess_data(json_path, images_dir)
            
            if image is None:
                continue  # 이미지 로드에 실패한 경우 다음 파일로 넘어갑니다.
            
            # 데이터 증강
            aug_image, aug_boxes, aug_labels = augment_data(image, boxes, labels)
            
            # YOLO 형식으로 변환
            yolo_labels = convert_to_yolo_format(aug_boxes, aug_labels, aug_image.shape[:2])
            
            # 증강된 이미지 저장
            aug_image_filename = f"aug_{json_file[:-5]}.jpg"
            cv2.imwrite(os.path.join(images_dir, aug_image_filename), cv2.cvtColor(aug_image, cv2.COLOR_RGB2BGR))
            
            # 증강된 레이블 저장
            aug_label_filename = f"aug_{json_file[:-5]}.txt"
            with open(os.path.join(labels_dir, aug_label_filename), 'w') as f:
                f.write('\n'.join(yolo_labels))
            
            data.append(aug_image_filename)
    
    return data

# 클래스 매핑 정의
class_mapping = {
    'car': 0, 'bus': 1, 'truck': 2, 'special vehicle': 3,
    'motorcycle': 4, 'bicycle': 5, 'personal mobility': 6,
    'person': 7, 'Traffic_light': 8, 'Traffic_sign': 9
}

# 메인 실행 코드
if __name__ == "__main__":
    # 현재 작업 디렉토리 출력
    print("Current working directory:", os.getcwd())
    
    # 데이터 준비
    train_data = prepare_dataset('training')
    val_data = prepare_dataset('validation')
    
    # 데이터 준비 결과 출력
    print(f"Prepared {len(train_data)} training samples")
    print(f"Prepared {len(val_data)} validation samples")

    # YAML 설정 파일 생성
    yaml_content = f"""
    train: training/images
    val: validation/images
    test: test/images

    nc: {len(class_mapping)}
    names: {list(class_mapping.keys())}
    """

    with open('dataset.yaml', 'w') as f:
        f.write(yaml_content)

    # 모델 로드 및 학습
    model = YOLO('yolov5s.pt')  # 사전 학습된 YOLOv5s 모델 로드

    # 모델 학습
    results = model.train(
        data='dataset.yaml',
        epochs=100,
        imgsz=640,
        batch=16,
        name='yolov5_autonomous_driving'
    )

    # 모델 평가
    results = model.val()

    # 테스트 데이터에 대한 예측
    test_results = model('test/images')

    # 결과 저장
    test_results.save('test_results')

Current working directory: /home/mira/Desktop/KistAIRobot/david/autonomous-driving/project/2DBB
Image file not found: training/images/S_DRG_230629_008_FC_049.jpg
Image file not found: training/images/S_DRG_230629_008_RL_069.jpg
Image file not found: training/images/S_DRG_230629_009_FL_058.jpg
Image file not found: training/images/S_DRG_230629_008_RL_033.jpg
Image file not found: training/images/S_DRG_230629_008_FC_104.jpg
Image file not found: training/images/S_DRG_230629_008_FL_106.jpg
Image file not found: training/images/S_DRG_230629_008_RL_106.jpg
Image file not found: training/images/S_DRG_230629_009_RR_108.jpg
Image file not found: training/images/S_DRG_230629_008_FL_040.jpg
Image file not found: training/images/S_DRG_230629_009_RL_085.jpg
Image file not found: training/images/S_DRG_230629_009_FR_111.jpg
Image file not found: training/images/S_DRG_230629_009_FC_114.jpg
Image file not found: training/images/S_DRG_230629_008_RR_055.jpg
Image file not found: training/images/S_DRG_23

OSError: [Errno 28] No space left on device

In [3]:
"""import os
import json
import torch
import cv2
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
import yolov5

# 사용자 정의 데이터셋 클래스
class AutonomousDrivingDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_dir = os.path.join(root_dir, 'images')
        self.label_dir = os.path.join(root_dir, 'labels')
        self.image_files = [f for f in os.listdir(self.image_dir) if f.endswith('.jpg')]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.image_dir, img_name)
        label_path = os.path.join(self.label_dir, img_name.replace('.jpg', '.json'))

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        with open(label_path, 'r') as f:
            label_data = json.load(f)

        boxes = []
        labels = []
        for annotation in label_data['Annotation']:
            x_min, y_min, x_max, y_max = annotation['data']
            boxes.append([x_min, y_min, x_max, y_max])
            labels.append(self.class_to_index(annotation['class_name']))

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)

        target = {}
        target['boxes'] = boxes
        target['labels'] = labels

        if self.transform:
            augmented = self.transform(image=image, bboxes=boxes, class_labels=labels)
            image = augmented['image']
            target['boxes'] = torch.tensor(augmented['bboxes'])
            target['labels'] = torch.tensor(augmented['class_labels'])

        return image, target

    @staticmethod
    def class_to_index(class_name):
        classes = ['car', 'bus', 'truck', 'special vehicle', 'motorcycle', 'bicycle', 
                   'personal mobility', 'person', 'Traffic_light', 'Traffic_sign']
        return classes.index(class_name)

# 데이터 증강 및 전처리
train_transform = A.Compose([
    A.Resize(height=640, width=640),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.MotionBlur(p=0.2),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))

# 데이터셋 및 데이터로더 생성
train_dataset = AutonomousDrivingDataset('2DBB/training', transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4, collate_fn=lambda x: tuple(zip(*x)))

# YOLOv5 모델 로드 및 설정
model = yolov5.load('yolov5s.pt')
model.train()

# 하이퍼파라미터 설정
hyp = {
    'lr0': 0.01,
    'lrf': 0.1,
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'warmup_epochs': 3.0,
    'warmup_momentum': 0.8,
    'warmup_bias_lr': 0.1,
    'box': 0.05,
    'cls': 0.5,
    'cls_pw': 1.0,
    'obj': 1.0,
    'obj_pw': 1.0,
    'iou_t': 0.20,
    'anchor_t': 4.0,
    'fl_gamma': 0.0,
    'hsv_h': 0.015,
    'hsv_s': 0.7,
    'hsv_v': 0.4,
    'degrees': 0.0,
    'translate': 0.1,
    'scale': 0.5,
    'shear': 0.0,
    'perspective': 0.0,
    'flipud': 0.0,
    'fliplr': 0.5,
    'mosaic': 1.0,
    'mixup': 0.0
}

# 모델 학습
results = model.train(data='data.yaml', epochs=100, imgsz=640, batch_size=16, hyp=hyp)

# 모델 저장
model.save('best_model.pt')

# 검증 데이터셋에 대한 성능 평가
val_dataset = AutonomousDrivingDataset('2DBB/validation', transform=train_transform)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4, collate_fn=lambda x: tuple(zip(*x)))

results = model.val(data='data.yaml', dataloader=val_loader)
print(results)"""

ModuleNotFoundError: No module named 'yolov5'

In [2]:
import os
import json
import cv2
import numpy as np
import torch
from ultralytics import YOLO
from albumentations import (
    Compose, RandomBrightnessContrast, HueSaturationValue, GaussNoise,
    HorizontalFlip, RandomCrop, Rotate, ShiftScaleRotate
)

# 데이터 로딩 및 전처리 함수
def load_and_preprocess_data(json_path, image_dir):
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    image_path = os.path.join(image_dir, data['image_name'])
    
    if not os.path.exists(image_path):
        print(f"Image file not found: {image_path}")
        return None, None, None, None
    
    image = cv2.imread(image_path)
    
    if image is None:
        print(f"Failed to load image: {image_path}")
        return None, None, None, None
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    annotations = data['Annotation']
    boxes = []
    labels = []
    for ann in annotations:
        boxes.append(ann['data'])
        labels.append(ann['class_name'])
    
    return image, np.array(boxes), labels, data['image_name']

# 데이터 증강 함수
def augment_data(image, boxes, labels):
    augmentations = Compose([
        RandomBrightnessContrast(p=0.5),
        HueSaturationValue(p=0.5),
        GaussNoise(p=0.3),
        HorizontalFlip(p=0.5),
        RandomCrop(height=image.shape[0], width=image.shape[1], p=0.3),
        Rotate(limit=10, p=0.3),
        ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=10, p=0.3)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})
    
    augmented = augmentations(image=image, bboxes=boxes, labels=labels)
    return augmented['image'], np.array(augmented['bboxes']), augmented['labels']

# YOLO 형식으로 레이블 변환
def convert_to_yolo_format(boxes, labels, image_size):
    yolo_labels = []
    for box, label in zip(boxes, labels):
        x_center = (box[0] + box[2]) / 2 / image_size[1]
        y_center = (box[1] + box[3]) / 2 / image_size[0]
        width = (box[2] - box[0]) / image_size[1]
        height = (box[3] - box[1]) / image_size[0]
        class_id = class_mapping[label]
        yolo_labels.append(f"{class_id} {x_center} {y_center} {width} {height}")
    return yolo_labels

# 데이터셋 준비 함수
def prepare_dataset(data_dir):
    images_dir = os.path.join(data_dir, 'images')
    labels_dir = os.path.join(data_dir, 'labels')
    
    data = []
    for json_file in os.listdir(labels_dir):
        if json_file.endswith('.json'):
            json_path = os.path.join(labels_dir, json_file)
            image, boxes, labels, image_name = load_and_preprocess_data(json_path, images_dir)
            
            if image is None:
                continue
            
            # 원본 이미지 및 레이블 처리
            orig_yolo_labels = convert_to_yolo_format(boxes, labels, image.shape[:2])
            orig_label_filename = f"{json_file[:-5]}.txt"
            with open(os.path.join(labels_dir, orig_label_filename), 'w') as f:
                f.write('\n'.join(orig_yolo_labels))
            
            data.append(os.path.join(images_dir, image_name))
            
            # 데이터 증강 및 처리 (저장하지 않고 메모리에서 처리)
            aug_image, aug_boxes, aug_labels = augment_data(image, boxes, labels)
            aug_yolo_labels = convert_to_yolo_format(aug_boxes, aug_labels, aug_image.shape[:2])
            
            # 증강된 데이터를 메모리에 저장
            data.append((aug_image, aug_yolo_labels))
    
    return data

# 클래스 매핑 정의
class_mapping = {
    'car': 0, 'bus': 1, 'truck': 2, 'special vehicle': 3,
    'motorcycle': 4, 'bicycle': 5, 'personal mobility': 6,
    'person': 7, 'Traffic_light': 8, 'Traffic_sign': 9
}

# 커스텀 데이터셋 클래스
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        if isinstance(item, tuple):  # 증강된 데이터
            image, labels = item
        else:  # 원본 이미지 경로
            image = cv2.imread(item)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            label_path = item.replace('images', 'labels').replace('.jpg', '.txt').replace('.png', '.txt')
            with open(label_path, 'r') as f:
                labels = f.read().splitlines()
        
        # 이미지와 레이블을 YOLO가 기대하는 형식으로 변환
        # (이 부분은 YOLO 모델의 요구사항에 따라 조정해야 할 수 있습니다)
        
        return image, labels

# 메인 실행 코드
if __name__ == "__main__":
    print("Current working directory:", os.getcwd())
    
    # 데이터 준비
    train_data = prepare_dataset('training')
    val_data = prepare_dataset('validation')
    
    print(f"Prepared {len(train_data)} training samples")
    print(f"Prepared {len(val_data)} validation samples")

    # 커스텀 데이터셋 생성
    train_dataset = CustomDataset(train_data)
    val_dataset = CustomDataset(val_data)

    # YAML 설정 파일 생성
    yaml_content = f"""
    train: training/images
    val: validation/images
    test: test/images

    nc: {len(class_mapping)}
    names: {list(class_mapping.keys())}
    """

    with open('dataset.yaml', 'w') as f:
        f.write(yaml_content)

    # 경량화된 모델 로드 (YOLOv5n)
    model = YOLO('yolov5n.pt')

    # 모델 학습
    results = model.train(
        data='dataset.yaml',
        epochs=20,
        imgsz=640,
        batch=16,
        save_period=2,  # 2 에폭마다 모델 저장
        weight_decay=0.0005,  # L2 정규화
        mosaic=0.5,  # 모자이크 증강
        mixup=0.2,  # Mixup 증강
        name='yolov5n_autonomous_driving'
    )

    # 모델 평가
    results = model.val()

    # 테스트 데이터에 대한 예측
    test_results = model('test/images')

    # 결과 저장
    test_results.save('test_results')

    # 모델 내보내기 (ONNX 포맷으로 경량화 및 양자화)
    model.export(format='onnx', simplify=True, opset=13, dynamic=True, half=True)

Current working directory: /home/mira/Desktop/KistAIRobot/david/autonomous-driving/project/2DBB
Image file not found: training/images/S_DRG_230629_008_FC_049.jpg
Image file not found: training/images/S_DRG_230629_008_RL_069.jpg
Image file not found: training/images/S_DRG_230629_009_FL_058.jpg
Image file not found: training/images/S_DRG_230629_008_RL_033.jpg
Image file not found: training/images/S_DRG_230629_008_FC_104.jpg
Image file not found: training/images/S_DRG_230629_008_FL_106.jpg
Image file not found: training/images/S_DRG_230629_008_RL_106.jpg
Image file not found: training/images/S_DRG_230629_009_RR_108.jpg
Image file not found: training/images/S_DRG_230629_008_FL_040.jpg
Image file not found: training/images/S_DRG_230629_009_RL_085.jpg
Image file not found: training/images/S_DRG_230629_009_FR_111.jpg
Image file not found: training/images/S_DRG_230629_009_FC_114.jpg
Image file not found: training/images/S_DRG_230629_008_RR_055.jpg
